In [1]:
import json
import os
import time
import random
from PIL import Image
from PIL.Image import BICUBIC
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from torchvision.datasets.cifar import CIFAR10
import ignite
from ignite.contrib.handlers import ProgressBar

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

# Arguments

In [2]:
pretrained = ''
resume = 'd:/log/efficientnet_c10/checkpoint.pth.tar'

In [3]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale
data_dir = 'd:/dataset/cifar10/cifar-10-batches-py'

# Optimization
num_classes = 10
epochs = 400
start_epoch = 0
train_batch = 60
test_batch = 50
lr = 0.1
schedule = [150, 275]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = 'd:/log/efficientnet_c10/' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

best_acc = 0
image_size = 224

GPU device 0: True


In [4]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [5]:
train_aug = transforms.Compose([
    transforms.Resize(image_size, BICUBIC),
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(image_size, BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = CIFAR10(data_dir, train=True, transform=train_aug, download=False)
test_dataset = CIFAR10(data_dir, train=False, transform=val_aug, download=False)

train_eval_indices = [random.randint(0, len(train_dataset) - 1) for i in range(len(test_dataset))]
train_eval_dataset = Subset(train_dataset, train_eval_indices)


# pin_memory : cuda pin memeory use
train_loader = DataLoader(train_dataset, batch_size=train_batch, drop_last=True, 
                          shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(test_dataset, batch_size=test_batch, drop_last=False,
                        shuffle=False, num_workers=num_workers, pin_memory=True)
eval_train_loader = DataLoader(train_eval_dataset, batch_size=train_batch, drop_last=False,
                              shuffle=False, num_workers=num_workers, pin_memory=True)

# Model

In [6]:
model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained))

In [7]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.02M


In [8]:
# summary(model, input_size=(3,64,64), device='cuda')

# Loss

In [9]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, nesterov=True)

In [10]:
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [11]:
# Resume
if resume:
    checkpoint_copy = checkpoint
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
    checkpoint = torch.load(resume)
    best_acc = checkpoint['best_acc']
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    checkpoint = checkpoint_copy
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

==> Resuming from checkpoint..


In [12]:
checkpoint

'd:/log/efficientnet_c10/'

# Train

In [13]:

def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [14]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [15]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [16]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)
    scheduler_warmup.step()
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [170 | 400] LR: 0.017000


C:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:82: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


1/833 Data:5.038 | Batch:7.992 | Total:0:00:08 | ETA:1:50:56 | Loss:0.07889077812433243 | top1:98.33332824707031
11/833 Data:0.040 | Batch:0.320 | Total:0:00:11 | ETA:0:15:07 | Loss:0.06489698656580666 | top1:98.0302963256836
21/833 Data:0.040 | Batch:0.322 | Total:0:00:14 | ETA:0:04:22 | Loss:0.0878033506728354 | top1:97.30158996582031
31/833 Data:0.041 | Batch:0.325 | Total:0:00:17 | ETA:0:04:20 | Loss:0.08882488634797835 | top1:97.0967788696289
41/833 Data:0.041 | Batch:0.317 | Total:0:00:21 | ETA:0:04:15 | Loss:0.0945938744741242 | top1:97.03251647949219
51/833 Data:0.041 | Batch:0.321 | Total:0:00:24 | ETA:0:04:11 | Loss:0.08977822734809973 | top1:97.18954467773438
61/833 Data:0.042 | Batch:0.325 | Total:0:00:27 | ETA:0:04:07 | Loss:0.0927583507307973 | top1:97.07650756835938
71/833 Data:0.040 | Batch:0.334 | Total:0:00:30 | ETA:0:04:09 | Loss:0.0909375322746559 | top1:97.18309783935547
81/833 Data:0.042 | Batch:0.324 | Total:0:00:33 | ETA:0:04:06 | Loss:0.08763099617014329 | top1

721/833 Data:0.040 | Batch:0.321 | Total:0:04:06 | ETA:0:00:38 | Loss:0.08485243389616758 | top1:97.15673065185547
731/833 Data:0.041 | Batch:0.335 | Total:0:04:09 | ETA:0:00:35 | Loss:0.08445061437458197 | top1:97.17053985595703
741/833 Data:0.040 | Batch:0.323 | Total:0:04:13 | ETA:0:00:31 | Loss:0.08454806507155349 | top1:97.16374206542969
751/833 Data:0.042 | Batch:0.350 | Total:0:04:16 | ETA:0:00:27 | Loss:0.08494315883944607 | top1:97.15711975097656
761/833 Data:0.040 | Batch:0.334 | Total:0:04:19 | ETA:0:00:27 | Loss:0.08516941846375269 | top1:97.14191436767578
771/833 Data:0.040 | Batch:0.330 | Total:0:04:23 | ETA:0:00:21 | Loss:0.0855018475902872 | top1:97.1292724609375
781/833 Data:0.041 | Batch:0.326 | Total:0:04:26 | ETA:0:00:18 | Loss:0.0856354468011401 | top1:97.11907958984375
791/833 Data:0.042 | Batch:0.324 | Total:0:04:30 | ETA:0:00:15 | Loss:0.0857011638932029 | top1:97.12389373779297
801/833 Data:0.040 | Batch:0.327 | Total:0:04:33 | ETA:0:00:11 | Loss:0.085925256256

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


...

501/833 Data:0.041 | Batch:0.349 | Total:0:02:47 | ETA:0:01:51 | Loss:0.04531253160020716 | top1:98.50299072265625
511/833 Data:0.063 | Batch:0.378 | Total:0:02:51 | ETA:0:01:49 | Loss:0.04516702665003753 | top1:98.49967193603516
521/833 Data:0.041 | Batch:0.322 | Total:0:02:54 | ETA:0:01:43 | Loss:0.04513789201300545 | top1:98.49008178710938
531/833 Data:0.056 | Batch:0.350 | Total:0:02:58 | ETA:0:01:42 | Loss:0.04488226554802445 | top1:98.49968719482422
541/833 Data:0.055 | Batch:0.409 | Total:0:03:01 | ETA:0:01:40 | Loss:0.044776094027828464 | top1:98.50585174560547
551/833 Data:0.042 | Batch:0.329 | Total:0:03:04 | ETA:0:01:37 | Loss:0.044415083228413546 | top1:98.51481628417969
561/833 Data:0.041 | Batch:0.333 | Total:0:03:08 | ETA:0:01:38 | Loss:0.04412473668935733 | top1:98.52346801757812
571/833 Data:0.040 | Batch:0.327 | Total:0:03:11 | ETA:0:01:26 | Loss:0.043913438213738776 | top1:98.53182220458984
581/833 Data:0.041 | Batch:0.321 | Total:0:03:14 | ETA:0:01:22 | Loss:0.04388

361/833 Data:0.045 | Batch:0.324 | Total:0:02:02 | ETA:0:02:38 | Loss:0.04590351434767639 | top1:98.4487533569336
371/833 Data:0.051 | Batch:0.393 | Total:0:02:05 | ETA:0:02:41 | Loss:0.04573232122518705 | top1:98.45911407470703
381/833 Data:0.041 | Batch:0.322 | Total:0:02:08 | ETA:0:02:29 | Loss:0.04518673629341085 | top1:98.47331237792969
391/833 Data:0.041 | Batch:0.323 | Total:0:02:12 | ETA:0:02:24 | Loss:0.044567088379769984 | top1:98.49531555175781
401/833 Data:0.041 | Batch:0.349 | Total:0:02:15 | ETA:0:02:24 | Loss:0.044611908349108664 | top1:98.48295593261719
411/833 Data:0.043 | Batch:0.335 | Total:0:02:18 | ETA:0:02:26 | Loss:0.04459772471263733 | top1:98.49554443359375
421/833 Data:0.040 | Batch:0.324 | Total:0:02:22 | ETA:0:02:26 | Loss:0.044142661041042586 | top1:98.51544189453125
431/833 Data:0.050 | Batch:0.362 | Total:0:02:25 | ETA:0:02:14 | Loss:0.04362254567863528 | top1:98.53054809570312
441/833 Data:0.041 | Batch:0.325 | Total:0:02:29 | ETA:0:02:17 | Loss:0.043690

231/833 Data:0.041 | Batch:0.334 | Total:0:01:20 | ETA:0:03:16 | Loss:0.05062029037887414 | top1:98.29725646972656
241/833 Data:0.043 | Batch:0.333 | Total:0:01:23 | ETA:0:03:13 | Loss:0.05129568984883468 | top1:98.28492736816406
251/833 Data:0.040 | Batch:0.322 | Total:0:01:26 | ETA:0:03:11 | Loss:0.05093170023509125 | top1:98.32006072998047
261/833 Data:0.040 | Batch:0.326 | Total:0:01:29 | ETA:0:03:06 | Loss:0.04987658080938174 | top1:98.34610748291016
271/833 Data:0.041 | Batch:0.325 | Total:0:01:33 | ETA:0:03:02 | Loss:0.049889432397536584 | top1:98.33948516845703
281/833 Data:0.041 | Batch:0.324 | Total:0:01:36 | ETA:0:02:59 | Loss:0.05000520233300828 | top1:98.3451919555664
291/833 Data:0.039 | Batch:0.322 | Total:0:01:39 | ETA:0:02:56 | Loss:0.050509929425501955 | top1:98.31615447998047
301/833 Data:0.041 | Batch:0.327 | Total:0:01:42 | ETA:0:02:53 | Loss:0.050757448149269115 | top1:98.2890396118164
311/833 Data:0.040 | Batch:0.331 | Total:0:01:46 | ETA:0:02:49 | Loss:0.0508240

101/833 Data:0.041 | Batch:0.325 | Total:0:00:39 | ETA:0:04:01 | Loss:0.043191033452380415 | top1:98.36634063720703
111/833 Data:0.041 | Batch:0.331 | Total:0:00:42 | ETA:0:04:01 | Loss:0.04221454904381152 | top1:98.40840911865234
121/833 Data:0.040 | Batch:0.325 | Total:0:00:45 | ETA:0:03:54 | Loss:0.04366949201977142 | top1:98.3057861328125
131/833 Data:0.041 | Batch:0.328 | Total:0:00:49 | ETA:0:03:49 | Loss:0.04384992170753668 | top1:98.30789184570312
141/833 Data:0.040 | Batch:0.323 | Total:0:00:52 | ETA:0:03:44 | Loss:0.043711369066641186 | top1:98.33333587646484
151/833 Data:0.041 | Batch:0.328 | Total:0:00:55 | ETA:0:03:43 | Loss:0.04297128837402996 | top1:98.36643981933594
161/833 Data:0.040 | Batch:0.330 | Total:0:00:58 | ETA:0:03:39 | Loss:0.04378475934470952 | top1:98.35403442382812
171/833 Data:0.041 | Batch:0.325 | Total:0:01:02 | ETA:0:03:39 | Loss:0.044973886204537614 | top1:98.32359313964844
181/833 Data:0.042 | Batch:0.327 | Total:0:01:05 | ETA:0:03:32 | Loss:0.045318

821/833 Data:0.040 | Batch:0.323 | Total:0:04:40 | ETA:0:00:04 | Loss:0.043127012886158014 | top1:98.44296264648438
831/833 Data:0.045 | Batch:0.339 | Total:0:04:43 | ETA:0:00:01 | Loss:0.042981394770707615 | top1:98.44364166259766
200/200 Data:0.001 | Batch:0.081 | Total:0:00:18 | ETA:0:00:00 | Loss:0.41313237239955924 | top1:91.82999420166016


In [17]:
logger.close()
print('Best acc:', best_acc)

Best acc: tensor(91.9900, device='cuda:0')
